In [1]:
#importing modules
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import collections
import statsmodels.stats.proportion
%matplotlib inline

In [2]:
data_xls = pd.read_excel('/Users/manueltanguma/Downloads/crimes.xls', index_col=3)
data_xls.to_csv('crimes.csv', encoding='utf-8')

In [3]:
#the target csv file
data = pd.read_csv('crimes.csv', encoding='latin-1')

In [4]:
data.head(10)

,Unnamed: 0,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NEW YORK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Offenses Known to Law Enforcement,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"by City, 2013",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Murder and\nnonnegligent\nmanslaughter,City,Population,Violent\ncrime,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
4,0,Adams Village,1861,0,NaN,0,0,0,12,2,10,0,0
5,0,Addison Town and Village,2577,3,NaN,0,0,3,24,3,20,1,0
6,0,Akron Village,2846,3,NaN,0,0,3,16,1,15,0,0
7,8,Albany,97956,791,NaN,30,227,526,4090,705,3243,142,NaN
8,0,Albion Village,6388,23,NaN,3,4,16,223,53,165,5,NaN
9,0,Alfred Village,4089,5,NaN,0,3,2,46,10,36,0,NaN


In [5]:
data = data.iloc[3:]

In [6]:
data.head(10)

,Unnamed: 0,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
3,Murder and\nnonnegligent\nmanslaughter,City,Population,Violent\ncrime,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
4,0,Adams Village,1861,0,NaN,0,0,0,12,2,10,0,0
5,0,Addison Town and Village,2577,3,NaN,0,0,3,24,3,20,1,0
6,0,Akron Village,2846,3,NaN,0,0,3,16,1,15,0,0
7,8,Albany,97956,791,NaN,30,227,526,4090,705,3243,142,NaN
8,0,Albion Village,6388,23,NaN,3,4,16,223,53,165,5,NaN
9,0,Alfred Village,4089,5,NaN,0,3,2,46,10,36,0,NaN
10,0,Allegany Village,1781,3,NaN,0,0,3,10,0,10,0,0
11,1,Amherst Town,118296,107,NaN,7,31,68,2118,204,1882,32,3
12,0,Amityville Village,9519,9,NaN,2,4,3,210,16,188,6,1


In [7]:
new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

In [8]:
data = data.reset_index(drop=True)

In [9]:
cols = data.columns.tolist()

In [10]:
columnsTitles = ['City', 'Population', 'Murder and\nnonnegligent\nmanslaughter','Violent\ncrime', 'Rape\n(revised\ndefinition)1',
                'Rape\n(legacy\ndefinition)2', 'Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary',
                'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3']

data = data.reindex(columns=columnsTitles)

In [11]:
data = data[:-3]

In [12]:
#addressing column names to make them more readable
data.columns = [
    'city', 'population', 'murder', 'violent_crime',
    'rape_revised_definition_1', 'rape_revised_definition_2', 'robbery', 'aggravated_assault',
    'property_crime', 'burglary', 'larceny_theft', 'motorvehicle_theft', 'arson']

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 13 columns):
city                         348 non-null object
population                   348 non-null object
murder                       348 non-null object
violent_crime                348 non-null object
rape_revised_definition_1    0 non-null object
rape_revised_definition_2    348 non-null object
robbery                      348 non-null object
aggravated_assault           348 non-null object
property_crime               348 non-null object
burglary                     348 non-null object
larceny_theft                348 non-null object
motorvehicle_theft           348 non-null object
arson                        187 non-null object
dtypes: object(13)
memory usage: 35.4+ KB


In [14]:
#rape_revised_definition_1 seems to be empty of data; therefore will delete the column from the dataframe
data = data.drop(['rape_revised_definition_1'], axis=1)

In [15]:
#re-addressing column names to make them more readable
data.columns = [
    'city', 'population', 'murder', 'violent_crime',
    'rape', 'robbery', 'aggravated_assault',
    'property_crime', 'burglary', 'larceny_theft', 'motorvehicle_theft', 'arson']

In [16]:
#when converted the excel file to csv was left with dtypes of "object", had to convert values to integer or float to
#carry out calculations
for i in range(0, len(data.columns)):
    data.iloc[:,i] = pd.to_numeric(data.iloc[:,i], errors='ignore')
    # errors='ignore' lets strings remain as 'non-null objects'


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 12 columns):
city                  348 non-null object
population            348 non-null int64
murder                348 non-null int64
violent_crime         348 non-null int64
rape                  348 non-null int64
robbery               348 non-null int64
aggravated_assault    348 non-null int64
property_crime        348 non-null int64
burglary              348 non-null int64
larceny_theft         348 non-null int64
motorvehicle_theft    348 non-null int64
arson                 187 non-null float64
dtypes: float64(1), int64(10), object(1)
memory usage: 32.7+ KB


In [18]:
data.head(5)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0.0
1,Addison Town and Village,2577,0,3,0,0,3,24,3,20,1,0.0
2,Akron Village,2846,0,3,0,0,3,16,1,15,0,0.0
3,Albany,97956,8,791,30,227,526,4090,705,3243,142,NaN
4,Albion Village,6388,0,23,3,4,16,223,53,165,5,NaN


In [19]:
data.tail(5)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson
343,Woodbury Town,10685,0,3,0,2,1,541,9,529,3,NaN
344,Woodridge Village,829,0,7,0,0,7,17,8,9,0,0.0
345,Woodstock Town,5931,0,2,0,0,2,58,13,45,0,NaN
346,Yonkers,199134,6,1036,25,390,615,2368,470,1662,236,10.0
347,Yorktown Town,36643,0,15,0,2,13,334,45,287,2,NaN


In [20]:
#fill in NaN of arson with average of column
data['arson'].mean()

1.8716577540106951

In [21]:
data['arson'].fillna((data['arson'].mean()), inplace=True)

In [22]:
data['arson'] = data['arson'].apply(np.int64)

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 12 columns):
city                  348 non-null object
population            348 non-null int64
murder                348 non-null int64
violent_crime         348 non-null int64
rape                  348 non-null int64
robbery               348 non-null int64
aggravated_assault    348 non-null int64
property_crime        348 non-null int64
burglary              348 non-null int64
larceny_theft         348 non-null int64
motorvehicle_theft    348 non-null int64
arson                 348 non-null int64
dtypes: int64(11), object(1)
memory usage: 32.7+ KB


In [24]:
data['population_squared'] = data['population']**2

In [25]:
data['propertycrime_noalpha'] = data['population']+data['population_squared']+data['murder']+data['robbery']

In [26]:
def func(x):
    if x == 0:
        return int('0')
    elif x > 0:
        return int('1')

data['robbery_cat'] = data['robbery'].apply(func)

In [27]:
data['murder_cat'] = data['murder'].apply(func)

In [28]:
data.head(5)

,city,population,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson,population_squared,propertycrime_noalpha,robbery_cat,murder_cat
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0,3463321,3465182,0,0
1,Addison Town and Village,2577,0,3,0,0,3,24,3,20,1,0,6640929,6643506,0,0
2,Akron Village,2846,0,3,0,0,3,16,1,15,0,0,8099716,8102562,0,0
3,Albany,97956,8,791,30,227,526,4090,705,3243,142,1,9595377936,9595476127,1,1
4,Albion Village,6388,0,23,3,4,16,223,53,165,5,1,40806544,40812936,1,0


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 16 columns):
city                     348 non-null object
population               348 non-null int64
murder                   348 non-null int64
violent_crime            348 non-null int64
rape                     348 non-null int64
robbery                  348 non-null int64
aggravated_assault       348 non-null int64
property_crime           348 non-null int64
burglary                 348 non-null int64
larceny_theft            348 non-null int64
motorvehicle_theft       348 non-null int64
arson                    348 non-null int64
population_squared       348 non-null int64
propertycrime_noalpha    348 non-null int64
robbery_cat              348 non-null int64
murder_cat               348 non-null int64
dtypes: int64(15), object(1)
memory usage: 43.6+ KB


In [30]:
#getting rid of 'city' and 'population' columns from the dataframe
data = data.drop(['city', 'population'] , axis=1)

In [31]:
data.head(5)

,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson,population_squared,propertycrime_noalpha,robbery_cat,murder_cat
0,0,0,0,0,0,12,2,10,0,0,3463321,3465182,0,0
1,0,3,0,0,3,24,3,20,1,0,6640929,6643506,0,0
2,0,3,0,0,3,16,1,15,0,0,8099716,8102562,0,0
3,8,791,30,227,526,4090,705,3243,142,1,9595377936,9595476127,1,1
4,0,23,3,4,16,223,53,165,5,1,40806544,40812936,1,0


In [32]:
data.shape

(348, 14)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 14 columns):
murder                   348 non-null int64
violent_crime            348 non-null int64
rape                     348 non-null int64
robbery                  348 non-null int64
aggravated_assault       348 non-null int64
property_crime           348 non-null int64
burglary                 348 non-null int64
larceny_theft            348 non-null int64
motorvehicle_theft       348 non-null int64
arson                    348 non-null int64
population_squared       348 non-null int64
propertycrime_noalpha    348 non-null int64
robbery_cat              348 non-null int64
murder_cat               348 non-null int64
dtypes: int64(14)
memory usage: 38.1 KB


In [34]:
cols = data.columns.tolist()

In [35]:
cols

['murder',
 'violent_crime',
 'rape',
 'robbery',
 'aggravated_assault',
 'property_crime',
 'burglary',
 'larceny_theft',
 'motorvehicle_theft',
 'arson',
 'population_squared',
 'propertycrime_noalpha',
 'robbery_cat',
 'murder_cat']

In [36]:
#rearranging columns to make propertycrime_noalpha the last column
#data = data[[c for c in data if c not in ['propertycrime_noalpha']]+['propertycrime_noalpha']]


In [37]:
#rearranging columns to make propertycrime_noalpha the last column
data = data[[c for c in data if c not in ['murder_cat']]+ ['murder_cat']]

In [38]:
data.head(5)

,murder,violent_crime,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motorvehicle_theft,arson,population_squared,propertycrime_noalpha,robbery_cat,murder_cat
0,0,0,0,0,0,12,2,10,0,0,3463321,3465182,0,0
1,0,3,0,0,3,24,3,20,1,0,6640929,6643506,0,0
2,0,3,0,0,3,16,1,15,0,0,8099716,8102562,0,0
3,8,791,30,227,526,4090,705,3243,142,1,9595377936,9595476127,1,1
4,0,23,3,4,16,223,53,165,5,1,40806544,40812936,1,0


In [39]:
X = data.iloc[:, 0:-1].values
y = data.iloc[:, -1].values

In [40]:
print (X)

[[          0           0           0 ...,     3463321     3465182
            0]
 [          0           3           0 ...,     6640929     6643506
            0]
 [          0           3           0 ...,     8099716     8102562
            0]
 ..., 
 [          0           2           0 ...,    35176761    35182692
            0]
 [          6        1036          25 ..., 39654349956 39654549486
            1]
 [          0          15           0 ...,  1342709449  1342746094
            1]]


In [41]:
print (y)

[0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0]


In [42]:
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [44]:
from sklearn.linear_model import LogisticRegression

In [45]:
VanillaLogRegress = LogisticRegression()
VanillaLogRegress.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [46]:
print('R² for the model:')
print(VanillaLogRegress.score(X_train, y_train))
origparams = np.append(VanillaLogRegress.coef_, VanillaLogRegress.intercept_)
print('\nParameter estimates for the model:')
print(origparams)

R² for the model:
0.167381974249

Parameter estimates for the model:
[  2.05116639e-19   6.22511464e-18   2.32366126e-19   2.51973603e-18
   3.26789585e-18  -2.24720057e-17   1.42854079e-18  -2.47192591e-17
   8.18712626e-19  -1.02897564e-19   3.82959004e-11   3.82937066e-11
  -1.67337441e-19  -3.88439741e-19]


### Lasso regression = L1
### Ridge regression = L2

In [47]:
Lasso_regression=LogisticRegression(penalty='l1', solver='liblinear')
Lasso_regression.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Please note that when solver=liblinear, the R2 is 1.0 however, when the solver=saga the R2 is 0.16.

In [48]:
print('R² for the model:')
print(Lasso_regression.score(X_train, y_train))
origparams1 = np.append(Lasso_regression.coef_, Lasso_regression.intercept_)
print('\nParameter estimates for the model:')
print(origparams1)

R² for the model:
1.0

Parameter estimates for the model:
[  7.55454041e+00   6.79367517e-03   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   2.02363518e-03  -6.47373456e-04
   0.00000000e+00   0.00000000e+00  -1.19996285e-10   1.70575516e-10
   0.00000000e+00  -4.64392645e+00]


In [49]:
Ridge_regression=LogisticRegression(penalty='l2', solver='lbfgs')
Ridge_regression.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

In [50]:
print('R² for the model:')
print(Ridge_regression.score(X_train, y_train))
origparams2 = np.append(Ridge_regression.coef_, Ridge_regression.intercept_)
print('\nParameter estimates for the model:')
print(origparams2)

R² for the model:
0.167381974249

Parameter estimates for the model:
[  4.29678567e-19   1.17839610e-17   4.20857354e-19   4.84782880e-18
   6.08559629e-18  -6.16877179e-17   7.08872003e-19  -6.36226479e-17
   1.22605794e-18  -2.93666437e-19   3.82975097e-11   3.82922225e-11
  -3.78847280e-19  -8.73567397e-19]


### In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?